# Cloud Inpainting on SEN3 Data
This notebook is a quick test to load, mask with clouds and then try and recover the data hidden by clouds using an inpainting DL model

In [1]:
import utils.prepare_dataset as predat
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from network.model import GenModel
from network.loss import lossL1


2024-06-20 01:11:31.334191: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-20 01:11:31.334677: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-20 01:11:31.338379: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-20 01:11:31.384215: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-20 01:11:32.224239: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

## Extracting the bands from the Xarrays

The goal here is to convert the data into a more tensorflow friendly data format

In [2]:
dsnp_c = np.moveaxis(np.load("Cloudy_DS.npy"),3,0)
dsnp = np.moveaxis(np.load("Ground_truth.npy"),3,0)

In [14]:
#Split into training and test set


train_ds = tf.data.Dataset.from_tensor_slices((dsnp_c[:800,:,:,:], dsnp[:800,:,:,:]))
test_ds = tf.data.Dataset.from_tensor_slices((dsnp_c[800:,:,:,:], dsnp[800:,:,:,:]))

In [15]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_ds = train_ds.shuffle(SHUFFLE_BUFFER_SIZE).batch(1)#.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE)

In [5]:
model = GenModel()
coarseNet = model.coarseNet
coarseNet.compile(optimizer='adam', loss=lossL1, metrics=['mean_absolute_error'])
print(coarseNet.summary())


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gated_conv2d (GatedConv2D)      │ (None, 256, 256, 32)   │        17,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gated_conv2d_1 (GatedConv2D)    │ (None, 128, 128, 64)   │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gated_conv2d_2 (GatedConv2D)    │ (None, 128, 128, 64)   │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gated_conv2d_3 (GatedConv2D)    │ (None, 64, 64, 128)    │       147,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gated_conv2d_4 (GatedConv2D)    │ (None, 64, 64, 128)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gated_conv2d_5 (GatedConv2D)    │ (None, 64, 64, 128)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gated_conv2d_6 (GatedConv2D)    │ (None, 64, 64, 128)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gated_conv2d_7 (GatedConv2D)    │ (None, 64, 64, 128)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gated_conv2d_8 (GatedConv2D)    │ (None, 64, 64, 128)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gated_conv2d_9 (GatedConv2D)    │ (None, 64, 64, 128)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gated_conv2d_10 (GatedConv2D)   │ (None, 64, 64, 128)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gated_de_conv2d (GatedDeConv2D) │ (None, 128, 128, 64)   │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gated_conv2d_12 (GatedConv2D)   │ (None, 128, 128, 64)   │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gated_de_conv2d_1               │ (None, 256, 256, 32)   │        65,600 │
│ (GatedDeConv2D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gated_conv2d_14 (GatedConv2D)   │ (None, 256, 256, 32)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gated_conv2d_15 (GatedConv2D)   │ (None, 256, 256, 32)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gated_conv2d_16 (GatedConv2D)   │ (None, 256, 256, 11)   │         6,358 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,758,806 (10.52 MB)

 Trainable params: 2,758,806 (10.52 MB)

 Non-trainable params: 0 (0.00 B)

None


In [22]:
np.max(dsnp_c[:,:,:,0])

nan

In [18]:
inp = train_ds.take(1)
out = coarseNet.predict(inp)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 660ms/step


In [7]:
#coarseNet.fit(train_ds, epochs=1, validation_data=test_ds)